In [10]:
import nltk   
nltk.download('vader_lexicon') #the nltk uses the pretrained model called Vader which has to be downloaded.
# Vader uses bag of words and looks at each word and scores it individually without considering the relationship between the words or context
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import emoji
import demoji
import spacy

from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import os


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/giu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
Dataset_name = 'TechLabsDataset.csv' 
DataDictionary_name = 'TechLabsDataset_Dictionary.csv'

Dataset = pd.read_csv(os.path.join('./Data/',Dataset_name), index_col = 0)
DataDictionary = pd.read_csv(os.path.join('./Data/',DataDictionary_name), index_col = 0)
Dataset.head()

,name,type_company,lat,lon,city,avg_stars_num,n_reviews_num,encoded_user,local_guide,clean_other_review_num,published_date,today_date,stars_num,review_EN,original_lang
0,Caretrex warehousing & logistics,Logistics service,51.593721,5.073492,Tilburg,5.0,2.0,HHpDM,0.0,4.0,2 years ago,2022-09-24,5.0,NaN,NaN
1,Caretrex warehousing & logistics,Logistics service,51.593721,5.073492,Tilburg,5.0,2.0,dYFWx,0.0,1.0,5 years ago,2022-09-24,5.0,NaN,NaN
2,FEFA Logistics,Trucking company,51.480074,5.446764,Eindhoven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dekkers Transport Holland,Trucking company,51.597249,5.027990,Tilburg,4.6,53.0,Pp1)c,0.0,5.0,a year ago,2022-09-24,5.0,Best employer ever. Worked there for approxima...,ENG
4,Dekkers Transport Holland,Trucking company,51.597249,5.027990,Tilburg,4.6,53.0,*nk1l,0.0,2.0,a year ago,2022-09-24,5.0,better place with very good people everything ...,ENG


In [12]:
review_df = pd.DataFrame(Dataset[~Dataset['review_EN'].isnull()]['review_EN'])
review_df.head()

,review_EN
3,Best employer ever. Worked there for approxima...
4,better place with very good people everything ...
5,Nice
6,Good firm. The staff are very welcoming and f...
7,"Top company, and coffee, so recommended\n\n"


In [13]:
# find emo
def find_emo(text):
    # find emoji and add to list
    all_emoji = demoji.findall(text)
    emo = ''
    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            # list emo
            emo = emo+','+all_emoji[j]
    return emo


# replace emo with empty text
def replace_emo(text):
    # find emo and replce with empty text
    all_emoji = demoji.findall(text)

    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            text = text.replace(j,'')
    return text

In [14]:
# add review with no emo
review_df['review_no_emo'] = review_df['review_EN'].apply(lambda x: replace_emo(x))

In [19]:
M1_vader = SentimentIntensityAnalyzer()# Vader Model
M2_distilBert = pipeline("sentiment-analysis")   #DistilBERT model

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

ValueError: Could not load model distilbert-base-uncased-finetuned-sst-2-english with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>).

In [ ]:
M2_distilBert(review_df['review_no_emo'].iloc[0].replace('\n',' '))

In [ ]:


compound_val_listVader = []
sentiment_listVader = []
sentiment_listPipeline = []
compound_val_listPipeline = []
for idx, review in with_reviews.iterrows():    #iterating through the df to get just the reviews
    review_text = review.loc['review_EN']
    emojis = emoji.distinct_emoji_list(review_text)    #Returns distinct list of emojis from the string
    for is_emoji in emojis:
        demoji_text = emoji.demojize(is_emoji)
        demoji_text = demoji_text.replace("_", " ")  # removing the underscore and the colon from the emoji text
        demoji_text = demoji_text.replace(":", "")
        review_text = review_text.replace(is_emoji, demoji_text)  # replacing the emoji with its text
        
    # Model 1
    sentiment = sia.polarity_scores(review_text) #using the nltk vader3 model to do the sentiment analysis 
    compound = sentiment['compound']   #taking the 'compound' key result
    compound_val_listVader.append(compound)
    if -0.1 < compound < 0.1:          # labelling the sentiments
        sentiment_listVader.append("neutral")
    elif compound > 0:
        sentiment_listVader.append("positive")
    else:
        sentiment_listVader.append("negative")

# Vader doesnt recognise some of the emoji text like thumbs up is analysed as neutral 
  
    # Model 2
    pipeline_result = sent_pipeline(review_text)[0]
    sentiment_listPipeline.append(pipeline_result["label"])
    compound_val_listPipeline.append(pipeline_result["score"])

#Adding columns for the compound sentiment value and the sentiment for both models
with_reviews["Sentiment_com"]= compound_val_listVader
with_reviews["Sentiment"]= sentiment_listVader
with_reviews["Sentiment_com_Pipeline"]= compound_val_listPipeline
with_reviews["Sentiment_Pipeline"]= sentiment_listPipeline
#print(with_reviews)


with_reviews.to_excel("Reviews_withsentiment.xlsx") #writing to excel